In [1]:
!pip3 install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 8.4MB/s 


In [0]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
# Load pre-trained model (weights)
model_version = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(model_version)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda(0)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

100%|██████████| 231508/231508 [00:00<00:00, 929886.07B/s]


# Generations

In [0]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx

In [0]:
# Generation modes as functions
import math
import time

def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]    
    
    return tokenize_batch(batch)

def parallel_sequential_generation_conditioned(seed_text, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True, sentiment=None):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    # untokenizing batch to add sentiment words
    untokenized_batch = untokenize_batch(batch)
    
    # tokenizing sentiment words
    sentiment_id = tokenizer.convert_tokens_to_ids([sentiment])[0]
    
    # inserting token at random position in each sentence of the batch
    ll = np.random.randint(0, max_len)
    for jj in range(batch_size):            
        batch[jj][seed_len+ll] = sentiment_id
        
    for ii in range(max_iter):
        
        # inserting mask at random position such that it doesn't replace the sentiment word
        kk = np.random.randint(0, max_len)
        while(kk == ll):
            kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
            
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]        
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(batch[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
            
    return untokenize_batch(batch)

def generate_conditioned(n_samples, seed_text="[CLS]", batch_size=10, max_len=25, 
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1, sentiment=None):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        batch = parallel_sequential_generation_conditioned(seed_text, max_len=max_len, top_k=top_k,
                                               temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False, sentiment=sentiment)
                
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch
    return sentences

In [0]:
# Utility functions

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))
    
def read_sents(in_file, should_detokenize=False):
    sents = [sent.strip().split() for sent in open(in_file).readlines()]
    if should_detokenize:
        sents = [detokenize(sent) for sent in sents]
    return sents

def write_sents(out_file, sents, should_detokenize=False):
    with open(out_file, "w") as out_fh:
        for sent in sents:
            sent = detokenize(sent[1:-1]) if should_detokenize else sent
            out_fh.write("%s\n" % " ".join(sent))

In [11]:
n_samples = 1000
batch_size = 20
max_len = 40
top_k = 100
temperature = 0.7

leed_out_len = 5 # max_len
burnin = 250
sample = True
max_iter = 500

# Choose the prefix context
seed_text = "[CLS]".split()
sentiment_lbl = 'happy'
for temp in [1.0]:
    bert_sents = generate_conditioned(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                          sample=sample, top_k=top_k, temperature=temp, burnin=burnin, max_iter=max_iter,
                          cuda=True, sentiment=sentiment_lbl)
    out_file = "/content/generated_sentences/%s-len%d-burnin%d-topk%d-temp%.3f-sentiment%s.txt" % (model_version, max_len, burnin, top_k, temp, sentiment_lbl)
    write_sents(out_file, bert_sents, should_detokenize=True)

Finished batch 1 in 43.344s
Finished batch 2 in 44.020s
Finished batch 3 in 43.898s
Finished batch 4 in 44.019s
Finished batch 5 in 43.932s
Finished batch 6 in 43.915s
Finished batch 7 in 43.933s
Finished batch 8 in 44.020s
Finished batch 9 in 43.971s
Finished batch 10 in 43.979s
Finished batch 11 in 43.935s
Finished batch 12 in 43.952s
Finished batch 13 in 44.026s
Finished batch 14 in 43.945s
Finished batch 15 in 44.030s
Finished batch 16 in 44.054s
Finished batch 17 in 44.033s
Finished batch 18 in 43.977s
Finished batch 19 in 43.950s
Finished batch 20 in 43.988s
Finished batch 21 in 44.001s
Finished batch 22 in 44.014s
Finished batch 23 in 43.946s
Finished batch 24 in 43.969s
Finished batch 25 in 43.987s
Finished batch 26 in 43.976s
Finished batch 27 in 43.979s
Finished batch 28 in 43.997s
Finished batch 29 in 44.034s
Finished batch 30 in 44.040s
Finished batch 31 in 43.990s
Finished batch 32 in 44.025s
Finished batch 33 in 43.967s
Finished batch 34 in 44.038s
Finished batch 35 in 44

In [0]:
in_file = "/content/generated_sentences/%s-len%d-burnin%d-topk%d-temp%.3f-sentiment%s.txt" % (model_version, max_len, burnin, top_k, temp, sentiment_lbl)
bert_sents = read_sents(in_file, should_detokenize=False)

In [13]:
for i in range(50):
    printer(bert_sents[i], should_detokenize=True)

here . jace ' s message was : think that you can stay here and think until they come . jace falling in love is too much . jace being happy for you is too much for me
satan ' ( taken from virgil as showing his fear of satan ) means evil . ' run away ' ( " stay away . run away " ) means happy as it is ( or ) soon
, also pregnant , hears the news of the newly family - run faith - based ministries , which she would like to continue : i ' ll never be happy . joan ( mary ) wilt
gloria " performed by mel clark . ( adapted from like a dove ) 2003 " just like a dove edition " of the original version of " like a happy dove " by bobby lavelli
never again ! " centuries and centuries and centuries and centuries - seeing a mastyr in another world tortured by a beautiful dragon twice his age to raise a happy child was one of her greatest nightmares
to keep these poor people in such high spirits . for them ever coming back . yeah , there it was ... but you could never stop feeling sad and happy about 